In [1]:
from helper_functions import *
import plotly.graph_objects as go
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
import wandb

In [2]:
from datetime import datetime
now = datetime.now()
datetime_str = now.strftime("%m/%d_%H:%M:%S")
datetime_str

'01/22_06:42:48'

In [3]:

data_df = pd.read_pickle("Kris_updated_yolov8.pkl")
data_df

,image name,car_count,timestamp_true
0,Kris_1206-1423.png,31,2023-06-12 14:23:00
1,Kris_1008-0950.png,32,2023-08-10 09:50:00
2,Kris_1108-0124.png,8,2023-08-11 01:24:00
3,Kris_2406-2128.png,6,2023-06-24 21:28:00
4,Kris_1006-0136.png,7,2023-06-10 01:36:00
...,...,...,...
963,Kris_1206-1252.png,24,2023-06-12 12:52:00
964,Kris_1006-1111.png,12,2023-06-10 11:11:00
965,Kris_0308-0805.png,0,2023-08-03 08:05:00
966,Kris_0806-2233.png,5,2023-06-08 22:33:00


In [4]:
data_df = preprocess_data(data_df)

In [5]:
# Load current experiment number
try:
    with open('experiment_counter.txt', 'r') as f:
        experiment_num = int(f.read())
except:
    experiment_num = 1

# Increment and save the updated experiment number
with open('experiment_counter.txt', 'w') as f:
    f.write(str(experiment_num + 1))

In [6]:
data_df.head()

,car_count,num_timestamp,month_no,month_name,day,day_of_week,day_of_week_name,is_weekend,hour_of_day,minutes,min_of_day,week_no,combined,centered_car_count
timestamp_true,,,,,,,,,,,,,,
2023-06-06 20:36:00,5,28101396,6,June,6,1,Tuesday,0,20,36,1236,1,06062036,-8.046488
2023-06-06 21:06:00,9,28101426,6,June,6,1,Tuesday,0,21,6,1266,1,06062106,-4.046488
2023-06-06 21:37:00,4,28101457,6,June,6,1,Tuesday,0,21,37,1297,1,06062137,-9.046488
2023-06-06 22:07:00,7,28101487,6,June,6,1,Tuesday,0,22,7,1327,1,06062207,-6.046488
2023-06-06 22:37:00,6,28101517,6,June,6,1,Tuesday,0,22,37,1357,1,06062237,-7.046488


In [7]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 968 entries, 2023-06-06 20:36:00 to 2023-08-14 13:26:00
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   car_count           968 non-null    int64  
 1   num_timestamp       968 non-null    int64  
 2   month_no            968 non-null    int32  
 3   month_name          968 non-null    object 
 4   day                 968 non-null    int32  
 5   day_of_week         968 non-null    int32  
 6   day_of_week_name    968 non-null    object 
 7   is_weekend          968 non-null    int64  
 8   hour_of_day         968 non-null    int32  
 9   minutes             968 non-null    int32  
 10  min_of_day          968 non-null    int32  
 11  week_no             968 non-null    int32  
 12  combined            968 non-null    object 
 13  centered_car_count  968 non-null    float64
dtypes: float64(1), int32(7), int64(3), object(3)
memory usage: 87.0+ KB


<h2> Exp 2: Num timestamp Revisited 

In [8]:
# as long as the time remains in a series, gaps will remain
# differences in the 

In [9]:
# Create a reference timestamp, which can be the first timestamp in the dataframe
reference_timestamp = data_df.index[0]

# Compute time deltas relative to the reference timestamp
data_df['time_delta'] = data_df.index - reference_timestamp


In [10]:
data_df['lag_time'] = data_df['num_timestamp'].diff()

In [11]:
data_df['lag_time'].unique()

array([       nan, 3.0000e+01, 3.1000e+01, 8.5100e+02, 2.5600e+02,
       8.3400e+02, 3.7000e+01, 4.3810e+03, 1.5520e+03, 1.0000e+00,
       1.9000e+01, 4.9000e+01, 1.0000e+01, 9.0000e+00, 2.4000e+01,
       1.5000e+01, 2.0000e+00, 4.0000e+01, 2.7000e+01, 4.3000e+01,
       6.0000e+01, 8.0000e+00, 3.8000e+01, 2.9000e+01, 2.5000e+01,
       3.5000e+01, 4.5000e+01, 1.8000e+01, 2.8000e+01, 3.9000e+01,
       5.2000e+01, 6.0000e+00, 3.0000e+00, 5.7000e+01, 5.9000e+01,
       5.3000e+01, 5.5000e+01, 2.1000e+01, 2.6000e+01, 4.0000e+00,
       5.0000e+00, 1.3000e+01, 1.4000e+01, 4.6000e+01, 1.1000e+01,
       1.6000e+01, 3.3000e+01, 5.8000e+01, 5.4000e+01, 7.0000e+00,
       1.7000e+01, 5.3810e+03, 4.4000e+01, 2.3000e+01, 2.2000e+01,
       3.4000e+01, 5.6000e+01, 4.8000e+01, 5.1000e+01, 4.7000e+01,
       3.2000e+01, 2.0000e+01, 4.1000e+01, 4.2000e+01, 5.2907e+04,
       1.2680e+03, 9.5000e+01, 2.7530e+03])

In [12]:
# Splitting the data into training (70%), validation (15%), and test (15%) sets
data_length = len(data_df)
train = data_df.iloc[:int(0.7 * data_length)]
valid = data_df.iloc[int(0.7 * data_length):int(0.8 * data_length)]
test = data_df.iloc[int(0.8 * data_length):]

X_train_norm  = train['num_timestamp'].values.reshape(-1,1)
y_train_norm  = train['centered_car_count']
X_valid_norm  = valid['num_timestamp'].values.reshape(-1,1)
y_valid_norm  = valid['centered_car_count']
X_test_norm  = test['num_timestamp'].values.reshape(-1,1)
y_test_norm  = test['centered_car_count']


In [13]:
# Splitting the data into training (70%), validation (15%), and test (15%) sets

short_data_df= data_df[:200]
data_length = len(short_data_df)
train = short_data_df.iloc[:int(0.7 * data_length)]
valid = short_data_df.iloc[int(0.7 * data_length):int(0.8 * data_length)]
test = short_data_df.iloc[int(0.8 * data_length):]

X_train_norm  = train['num_timestamp'].values.reshape(-1,1)
y_train_norm  = train['centered_car_count']
X_valid_norm  = valid['num_timestamp'].values.reshape(-1,1)
y_valid_norm  = valid['centered_car_count']
X_test_norm  = test['num_timestamp'].values.reshape(-1,1)
y_test_norm  = test['centered_car_count']


In [14]:
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel, Matern as C
from sklearn.gaussian_process.kernels import Matern

k1 = 1.0 * RationalQuadratic(length_scale=1, alpha=0.5, alpha_bounds=(1e-5, 1e5))

# # White kernel to account for noise
# k2 = WhiteKernel(noise_level=0.2, noise_level_bounds=(1e-10, 1e+1))

# #Periodic kernel
# k3 = ExpSineSquared(length_scale=0.9,
#                     length_scale_bounds=(1e-1, 10) ,
#                     periodicity=0.2,
#                     periodicity_bounds= (1e-1, 10))

# RBF kernel with a length scale of 1.0
# k4 = 10.0 * RBF(length_scale=1, length_scale_bounds=(1e-1, 1e4))

# k5 = 1.0 * Matern(length_scale=1.0, nu=2)

# Combined kernel
kernel = k1 


gpr = GaussianProcessRegressor(kernel=kernel, random_state= 42, n_restarts_optimizer= 10)


In [15]:
# properly initiate a wandb exp run
# plot_gpr_samples_plotly(gpr, 5, X_train_norm, y_train_norm)


In [16]:

# y_pred, y_std, gpr_model = gpr_train( gpr, X_train_norm ,y_train_norm, X_test_norm, y_test_norm)

In [17]:
def split_and_aggregate(data_df, features, target, agg_func='mean', train_ratio=0.6, valid_ratio=0.2):
    """
    Prepares time series data for training a model. 
    The function splits the original data into train, validate, and test subsets, 
    aggregates them based on the specified function, and returns the X and y splits for each subset.
    
    Parameters:
    - data_df: DataFrame containing the time series data
    - features: list of feature column names
    - target: target column name
    - aggregation_func: function to use for aggregation, default is 'mean'
    - train_ratio: fraction of data to be used for training
    - valid_ratio: fraction of data to be used for validation
    
    Returns:
    - X_train, y_train, X_valid, y_valid, X_test, y_test
    """
    # filter features 
    data_df = data_df[['car_count', 'month_no', 'day', 'day_of_week', 'hour_of_day', 'min_of_day', 'centered_car_count','combined']]
    
    # Calculating the split indices
    train_idx = int(train_ratio * len(data_df))
    valid_idx = train_idx + int(valid_ratio * len(data_df))
    
    # Splitting the original dataframe
    train_df = data_df.iloc[:train_idx]
    valid_df = data_df.iloc[train_idx:valid_idx]
    test_df = data_df.iloc[valid_idx:]
    
    # Aggregate dataframes
    if agg_func == 'mean':
        train_agg = train_df.groupby(features).mean().reset_index()
        valid_agg = valid_df.groupby(features).mean().reset_index()
        test_agg = test_df.groupby(features).mean().reset_index()
        
    # Separate X and y
    X_train = train_agg[features].values.reshape(-1, 1)
    y_train = train_agg[target].values
    X_valid = valid_agg[features].values.reshape(-1, 1)
    y_valid = valid_agg[target].values
    X_test = test_agg[features].values.reshape(-1, 1)
    y_test = test_agg[target].values
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test, train_agg, valid_agg,test_agg


In [18]:
train_ratio=0.6
valid_ratio=0.2
# filter features 
data_df = data_df[['car_count', 'month_no', 'day', 'day_of_week', 'hour_of_day', 'min_of_day', 'centered_car_count','combined']]

# Calculating the split indices
train_idx = int(train_ratio * len(data_df))
valid_idx = train_idx + int(valid_ratio * len(data_df))

# Splitting the original dataframe
train_df = data_df.iloc[:train_idx]
valid_df = data_df.iloc[train_idx:valid_idx]
test_df = data_df.iloc[valid_idx:]

In [19]:
def time_lagged_cross_validation(gpr, X_train, y_train, X_valid, y_valid, step=1):
    """
    Perform time-lagged cross-validation on given train and validate data.
    
    Parameters:
    - gpr: Gaussian Process Regressor model to be trained
    - X_train: Training data features
    - y_train: Training data target
    - X_valid: Validation data features
    - y_valid: Validation data target
    - step: step size for sliding window
    
    Returns:
    - List of metrics (e.g., MAE) for each validation step
    """
    metrics_list = []
    
    # Determine the total size of train and validation data
    total_size = X_train.shape[0] + X_valid.shape[0]
    
    # Define a function to compute the desired metric, e.g., MAE
    def compute_metric(y_true, y_pred):
        return mean_absolute_error(y_true, y_pred)
    
    for end_train_idx in range(X_train.shape[0], total_size, step):
        # Split data into current train and validation subsets
        current_X_train = X_train[:end_train_idx]
        current_y_train = y_train[:end_train_idx]
        
        start_valid_idx = end_train_idx - X_train.shape[0]
        current_X_valid = X_valid[start_valid_idx : start_valid_idx + step]
        current_y_valid = y_valid[start_valid_idx : start_valid_idx + step]
        
        # Train the GPR model
        gpr.fit(current_X_train, current_y_train)
        
        # Predict on the current validation set
        y_pred = gpr.predict(current_X_valid)
        
        # Compute the metric and append to the list
        metric_value = compute_metric(current_y_valid, y_pred)
        metrics_list.append(metric_value)
        
    return metrics_list


<h2>Exp 4: Agg on hour_of_day

In [20]:
X_train, y_train, X_valid, y_valid, X_test, y_test ,train_agg, valid_agg,test_agg = split_and_aggregate(data_df, 'hour_of_day', 'centered_car_count', agg_func='mean',train_ratio=0.6,valid_ratio=0.2)


In [21]:
train_agg

,hour_of_day,car_count,month_no,day,day_of_week,min_of_day,centered_car_count,combined
0,0,6.913043,6.173913,14.608696,3.043478,29.695652,-6.133444,2639133939434948134801219005117734675746536887...
1,1,7.272727,6.181818,14.318182,3.409091,87.500000,-5.773760,2759140027591541150359942353162167688692007543...
2,2,6.250000,6.166667,14.625000,2.750000,149.291667,-6.796488,2529253775292662827217706711628852609835290231...
3,3,10.142857,6.190476,13.809524,3.285714,208.142857,-2.903630,2890623838430190909436371856221651458299178975...
4,4,14.750000,6.200000,13.250000,3.150000,268.400000,1.703512,3035205030352205476883158001889859255780948298...
5,5,15.047619,6.190476,13.571429,3.047619,329.380952,2.001131,2890719552716862265115357476170864274009233811...
6,6,13.476190,6.190476,13.523810,3.333333,390.285714,0.429703,2890767171764962370924538395179032877455765854...
7,7,11.090909,6.181818,14.363636,3.136364,450.818182,-1.955579,2759414573048827427047763386652591941903061788...
8,8,15.333333,6.190476,13.523810,3.333333,510.095238,2.286846,2890863362242109371576233654256192570118644576...
9,9,19.720000,6.160000,14.360000,2.760000,568.600000,6.673512,2428365224283776168776153520468601028980832495...


In [22]:
valid_agg

,hour_of_day,car_count,month_no,day,day_of_week,min_of_day,centered_car_count,combined
0,0,1.857143,8.0,5.428571,4.428571,26.714286,-11.189345,1148572868628628694076549069603277755032768983...
1,1,3.000000,8.0,5.750000,4.000000,83.500000,-10.046488,1005013760050175045522963013077515641449158701...
2,2,2.750000,8.0,5.750000,4.000000,143.875000,-10.296488,1005026260050300143630950962302295588087775110...
3,3,2.000000,8.0,5.750000,4.000000,204.125000,-11.046488,1005038760050426383537093076294980381355167080...
4,4,3.375000,8.0,5.750000,4.000000,264.250000,-9.671488,1005051385050551338490614669830396309726434695...
5,5,4.875000,8.0,5.750000,4.000000,324.500000,-8.171488,1005063885050676322418787202578385771702173549...
6,6,4.625000,8.0,5.750000,4.000000,384.875000,-8.421488,1005076385050801306346959735326375233677912402...
7,7,7.375000,8.0,5.750000,4.000000,445.125000,-5.671488,1005088885050927546253101849319060026945304371...
8,8,8.375000,8.0,5.750000,4.000000,505.375000,-4.671488,1005101510051052615386438859331266439979449543...
9,9,9.750000,8.0,5.750000,4.000000,565.500000,-3.296488,1005114010051177599314611392079255901955188396...


In [23]:
test_agg

,hour_of_day,car_count,month_no,day,day_of_week,min_of_day,centered_car_count,combined
0,0,6.625000,8.0,11.000000,2.250000,32.500000,-6.421488,1011250760112545110254861439571103902995548054...
1,1,5.100000,8.0,11.400000,3.000000,92.500000,-7.946488,8090106080901360100986303457935264613138003813...
2,2,5.500000,8.0,11.400000,3.000000,152.600000,-7.546488,8090206080902361205041350356888848855791240858...
3,3,5.222222,8.0,11.444444,2.888889,210.777778,-7.824265,8989230089892633553799202900887185891560204058...
4,4,6.000000,8.0,11.500000,2.750000,271.750000,-7.046488,1011300885113046387150685629071974033248562902...
5,5,8.500000,8.0,11.500000,2.750000,331.875000,-4.546488,1011313385113171371078858161819963495224301755...
6,6,12.375000,8.0,11.500000,2.750000,392.000000,-0.671488,1011325885113296355007030694567952957200040609...
7,7,11.375000,8.0,11.500000,2.750000,452.500000,-1.671488,1011338510113422565938521869348064216862961340...
8,8,14.000000,8.0,11.500000,2.750000,512.750000,0.953512,1011351010113547549866694402096053678838700193...
9,9,14.500000,8.0,11.500000,2.750000,573.000000,1.453512,1011363510113672647974682351320833625477316603...


In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= train_agg['hour_of_day'],
                         y=train_agg['centered_car_count'],
                         mode='markers+lines',
                         name="Training "))
fig.add_trace(go.Scatter(x=valid_agg['hour_of_day'],
                         y=valid_agg['centered_car_count'],
                         mode='markers+lines',
                         name='Validate'))
fig.add_trace(go.Scatter(x=test_agg['hour_of_day'],
                         y=test_agg['centered_car_count'],
                         mode='markers+lines',
                         name='Test'))

# Update layout for better visualization
fig.update_layout(title='Standard train, validate and test subsets',
                  xaxis_title='Hours',
                  yaxis_title='Count',
                  legend_title='Subsets')

In [25]:
data_df_day = data_df[['car_count', 'month_no', 'day', 'day_of_week', 'hour_of_day', 'min_of_day', 'centered_car_count','combined']]

day_mean_df = data_df_day.groupby('hour_of_day').mean().reset_index()
day_med_df = data_df_day.groupby('hour_of_day').median().reset_index()

In [26]:

# # Create a new figure
# fig = go.Figure()

# # Add a trace for day_mean_df's centered_car_count
# fig.add_trace(go.Scatter(x=day_mean_df['hour_of_day'], 
#                          y=day_mean_df['centered_car_count'], 
#                          mode='markers+lines', 
#                          name='Mean Centered Car Count'))



# # Add a trace for day_med_df's centered_car_count
# fig.add_trace(go.Scatter(x=day_med_df['hour_of_day'], 
#                          y=day_med_df['centered_car_count'], 
#                          mode='markers+lines', 
#                          name='Median Centered Car Count'))

# # Update layout for better visualization
# fig.update_layout(title='Comparison of Mean and Median Aggregations',
#                   xaxis_title='Hour of Day',
#                   yaxis_title='Count',
#                   legend_title='Aggregations')

# fig.show()


In [27]:
ad_fuller_test(day_mean_df['centered_car_count'])
# ad_fuller_test(month_med_df['centered_car_count'])

ADF Statistic: -1.8307958779034694
p-value: 0.36528309622415844
Critical Values: {'1%': -4.01203360058309, '5%': -3.1041838775510207, '10%': -2.6909873469387753}
The series is not stationary.


In [28]:
X_hour_avg =  day_mean_df['hour_of_day'].values.reshape(-1,1)
y_hour_avg = day_mean_df['centered_car_count'].values.reshape(-1,1)

In [29]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared, ConstantKernel as C

# RBF kernel with a length scale of 1.0
k1 = 1 * RBF(length_scale=0.5, length_scale_bounds=(1e-3, 1e3))

# White kernel to account for noise
k2 = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))

# Periodic kernel
k3 = ExpSineSquared(length_scale=1.0,
                    length_scale_bounds=(1e-1, 50) ,
                    periodicity=1,
                    periodicity_bounds= (1e-1, 50))

# Combined kernel
kernel = k1 + k2 + k3

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer= 10)

In [30]:
# start a new wandb run to track this script

run = wandb.init(
    # set the wandb project where this run will be logged
    project="GPR",
    name= f"hour_of_day_agg_4.{experiment_num}_{datetime_str}"
)
y_pred, y_std = gpr_train(gpr=gpr,n_samples= 5,x_train=X_hour_avg,y_train=y_hour_avg, run=run)
# log_data(gpr_score_card(gpr, y_month_mean, y_std))
wandb.finish()

AttributeError: module 'wandb' has no attribute 'init'

<h2> Exp 5 : Rolling statistics

In [51]:
data_df['rw_hrs'] = data_df['centered_car_count'].rolling(window=5).mean()

/tmp/ipykernel_18040/2488354281.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
fig = go.Figure()
# Original data
fig.add_trace(go.Scatter(x=data_df.index, y=data_df['centered_car_count'], mode='lines', name='Original Data'))
fig.add_trace(go.Scatter(x=data_df.index, y=data_df['rw_hrs'], mode='lines', name='Rolling'))  
fig.update_layout(title='Rolling Statistics Plot', xaxis_title='Timestamp', yaxis_title='Car Count', height=800)
fig.show()

<h2> Exp 6: Agg on month

In [68]:
data_df_trunc = data_df[['car_count', 'month_no', 'day', 'day_of_week', 'hour_of_day', 'min_of_day', 'centered_car_count','combined']]

month_mean_df = data_df_trunc.groupby('day').mean().reset_index()
month_med_df = data_df_trunc.groupby('day').median().reset_index()

KeyError: "['day_of_week_name'] not in index"

In [54]:
ad_fuller_test(month_mean_df['centered_car_count'])
# ad_fuller_test(month_med_df['centered_car_count'])

ADF Statistic: -1.277011587326327
p-value: 0.6396228286558957
Critical Values: {'1%': -4.223238279489106, '5%': -3.189368925619835, '10%': -2.729839421487603}
The series is not stationary.


In [55]:
month_mean_df['day'].unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 17, 18, 19, 20,
       24, 25, 26], dtype=int32)

In [56]:
# Create a new figure
fig = go.Figure()



# Add a trace for day_mean_df's centered_car_count
fig.add_trace(go.Scatter(x=month_mean_df['day'], 
                         y=month_mean_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Mean Centered Car Count'))


# Add a trace for day_med_df's centered_car_count
fig.add_trace(go.Scatter(x=month_med_df['day'], 
                         y=month_med_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Median Centered Car Count'))

# Update layout for better visualization
fig.update_layout(title='Comparison of Mean and Median Aggregations by Month',
                  xaxis_title='Day of Month',
                  yaxis_title='Count',
                  legend_title='Aggregations')

fig.show()


In [57]:
X_month_mean=  month_mean_df['day'].values.reshape(-1,1)
y_month_mean = month_mean_df['centered_car_count'].values.reshape(-1,1)

In [58]:
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ExpSineSquared
from sklearn.gaussian_process.kernels import (RationalQuadratic,Exponentiation)

# RBF or SE kernel with a length scale of 1.0
k1 = 10 * RBF(length_scale=0.3, length_scale_bounds=(1e-4, 1e4))

# # White kernel to account for noise
# k2 = WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+20))

# Periodic kernel
k3 = ExpSineSquared(length_scale=1.0,
                    length_scale_bounds=(1e-1,1e02 ) ,
                    periodicity=0.2,
                    periodicity_bounds= (1e-1, 1e02))

k4 = 1.0 * RationalQuadratic(length_scale=0.5, alpha=0.5, alpha_bounds=(1e-03, 1e02))

# ExponentiatedQuadratic -> locally periodic exponentiation(quadratic)
#k5 = Exponentiation(Quadratic(), exponent= 2)


# Combined kernel:
kernel = k1 * k3

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer= 10)

In [40]:
# start a new wandb run to track this script

run = wandb.init(
    # set the wandb project where this run will be logged
    project="GPR",
    name= f"month_agg_6.{experiment_num}_{datetime_str}"
)
y_pred, y_std = gpr_train(gpr=gpr,n_samples= 5,x_train=X_month_mean,y_train=y_month_mean, run=run)
# log_data(gpr_score_card(gpr, y_month_mean, y_std))
wandb.finish()

AttributeError: module 'wandb' has no attribute 'init'

In [59]:

# gpr.fit(X_month_mean ,y_month_mean)

# y_pred, y_std = gpr.predict(X_month_mean, return_std=True)

# plot_gpr_samples_plotly(gpr, 5, X_month_mean,y_month_mean)
# log_data(gpr_score_card(gpr, y_month_mean, y_std))



<h2> Exp 6: Agg on Week

In [60]:
week_mean_df = data_df_trunc.groupby('day_of_week').agg({'centered_car_count':'mean'}).reset_index()
week_med_df = data_df_trunc.groupby('day_of_week').agg({'centered_car_count':'median'}).reset_index()

In [61]:
X_week_mean=  week_mean_df['day_of_week'].values.reshape(-1,1)
y_week_mean = week_mean_df['centered_car_count'].values.reshape(-1,1)

In [62]:
X_week_mean

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6]], dtype=int32)

In [63]:
ad_fuller_test(week_mean_df['centered_car_count'])
# ad_fuller_test(week_med_df['centered_car_count'])

ADF Statistic: -1.2315497591578144
p-value: 0.6598787282104795
Critical Values: {'1%': -5.354256481481482, '5%': -3.6462381481481483, '10%': -2.901197777777778}
The series is not stationary.


In [64]:
# Create a new figure
fig = go.Figure()

# Add a trace for day_mean_df's centered_car_count
fig.add_trace(go.Scatter(x=week_mean_df['day_of_week'], 
                         y=week_mean_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Mean Centered Car Count'))


# Add a trace for day_med_df's centered_car_count
fig.add_trace(go.Scatter(x=week_med_df['day_of_week'], 
                         y=week_med_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Median Centered Car Count'))

# Update layout for better visualization
fig.update_layout(title='Comparison of Mean and Median Aggregations by Week',
                  xaxis_title='Day of Week',
                  yaxis_title='Count',
                  legend_title='Aggregations')

fig.show()

In [65]:

# Create a new figure
fig = go.Figure()

# Add a trace for day_mean_df's centered_car_count
fig.add_trace(go.Scatter(x=day_mean_df['hour_of_day'], 
                         y=day_mean_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Hourly Mean Centered Car Count'))



# Add a trace for day_med_df's centered_car_count
fig.add_trace(go.Scatter(x=day_med_df['hour_of_day'], 
                         y=day_med_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Hourly Median Centered Car Count'))


# Add a trace for day_mean_df's centered_car_count
fig.add_trace(go.Scatter(x=month_mean_df['day'], 
                         y=month_mean_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Day of month Mean Centered Car Count'))


# Add a trace for day_med_df's centered_car_count
fig.add_trace(go.Scatter(x=month_med_df['day'], 
                         y=month_med_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Day of month Median Centered Car Count'))


# Add a trace for day_mean_df's centered_car_count
fig.add_trace(go.Scatter(x=week_mean_df['day_of_week'], 
                         y=week_mean_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Week Mean Centered Car Count'))


# Add a trace for day_med_df's centered_car_count
fig.add_trace(go.Scatter(x=week_med_df['day_of_week'], 
                         y=week_med_df['centered_car_count'], 
                         mode='markers+lines', 
                         name='Week Median Centered Car Count'))

# Update layout for better visualization
fig.update_layout(title='Comparison of Mean and Median Aggregations by Week',
                  xaxis_title='Days of Month',
                  yaxis_title='Count',
                  legend_title='Aggregations')

fig.show()

In [67]:
week_mean_df

,day_of_week,centered_car_count
0,0,3.791748
1,1,6.775430
2,2,2.488235
3,3,1.230108
4,4,-10.117916
5,5,-3.478306
6,6,-4.368096


In [66]:
from plotly.subplots import make_subplots

# Create a 3-row subplot figure
fig = make_subplots(rows=3, cols=1, 
                    subplot_titles=('Hourly Aggregation', 'Daily Aggregation', 'Weekly Aggregation'),
                    vertical_spacing=0.1)

# Define colors for consistency
color_mean = 'blue'
color_median = 'red'

# Hourly aggregation
fig.add_trace(go.Scatter(x=day_mean_df['hour_of_day'], 
                         y=day_mean_df['centered_car_count'], 
                         mode='lines', 
                         name=' Mean',
                         legendgroup= 'mean',
                         line=dict(color=color_mean)), row=1, col=1)

fig.add_trace(go.Scatter(x=day_med_df['hour_of_day'], 
                         y=day_med_df['centered_car_count'], 
                         mode='lines',
                         legendgroup="median" ,
                         name='Median',
                         line=dict(color=color_median, dash='dash')), row=1, col=1)

# Daily aggregation
fig.add_trace(go.Scatter(x=month_mean_df['day'], 
                         y=month_mean_df['centered_car_count'], 
                         mode='lines', 
                        legendgroup= 'mean',
                         showlegend= False,
                         name='Day of Month Mean',
                         line=dict(color=color_mean)), row=2, col=1)

fig.add_trace(go.Scatter(x=month_med_df['day'], 
                         y=month_med_df['centered_car_count'], 
                         mode='lines',
                         legendgroup="median" ,
                         name='Day of Month Median',
                          showlegend= False,
                         line=dict(color=color_median, dash='dash')), row=2, col=1)

# Weekly aggregation
fig.add_trace(go.Scatter(x=week_mean_df['day_of_week'], 
                         y=week_mean_df['centered_car_count'], 
                         mode='lines',
                        legendgroup= 'mean' ,
                         name='Week Mean',
                          showlegend= False,
                         line=dict(color=color_mean)), row=3, col=1)

fig.add_trace(go.Scatter(x=week_med_df['day_of_week_name'], 
                         y=week_med_df['centered_car_count'], 
                         mode='lines', 
                         legendgroup="median",
                         name='Week Median',
                          showlegend= False,
                         line=dict(color=color_median, dash='dash')), row=3, col=1)

# Labeling x-axes for each subplot
fig.update_xaxes(title_text="Hours of Day", row=1, col=1)
fig.update_xaxes(title_text="Days of Month", row=2, col=1)
fig.update_xaxes(title_text="Days of Week", row=3, col=1)

# Labeling y-axes for each subplot
# If you want individual y-axis titles:
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Count", row=3, col=1)

# Add a combined y-axis label using an annotation
fig.update_layout(
    legend_title='Aggregations',
    title='Comparison of Mean and Median Aggregations',
    height= 1000,
   
    annotations=[dict(text="Count",
                                    xref="paper", yref="paper",
                                    x=-0.05, y=0.5, showarrow=False,
                                    font=dict(size=14))]
                  )

fig.show()


KeyError: 'day_of_week_name'